# Test search utils


In [1]:
import yaml
import inspect
import importlib
import os,sys
import importlib
import arxiv

sys.path.append('..')

from model_discovery.evolution import BuildEvolution
import model_discovery.utils as U
import requests
import time
from tqdm import tqdm

headers={"X-API-KEY": os.environ['S2_API_KEY']}
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}'


strparams=[
    f"evoname=test_evo_000",
    "scales=14M,31M,70M",
    "selection_ratio=0.25",
    "select_method=random",
    "design_budget=0",
]
evo_system = BuildEvolution(
    strparams=';'.join(strparams),
    do_cache=False,
    # cache_type='diskcache',
)

ptree = evo_system.ptree


c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


2024-09-12:11:47:12,588 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-12:11:47:13,292 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-12:11:47:13,554 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


{'evoname': 'test_evo_000', 'scales': '14M,31M,70M', 'selection_ratio': '0.25', 'select_method': 'random', 'design_budget': '0'}
Evolution system initialized with scales: ['14M', '31M', '70M']
Current scale: 0
Budgets remaining: {'70M': 1, '31M': 4, '14M': 16}
Checkpoint directory: C:\ChengJunyan1\Research\model_discovery\ckpt\test_evo_000
gpt2 tree loaded


2024-09-12:11:47:14,495 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-12:11:47:15,211 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-12:11:47:15,486 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


## Get paper files

In [48]:

refs_s2ids={}
REF_TYPES=['Reference','ReferenceCore','ReferenceWithCode','ReferenceCoreWithTree']

for i in ptree.G.nodes:
    node=ptree.G.nodes[i]['data']
    if node.type in REF_TYPES:
        s2id=node.s2id
        if s2id is not None:
            refs_s2ids[i]=s2id
        else:
            print(i)

refs=ptree.filter_by_type(REF_TYPES)

feedbackmem
kangpt
ntk
s4pp
srt


### Get rec and files

In [54]:


def get_rec_papers(s2id):
    url=f'https://api.semanticscholar.org/recommendations/v1/papers/forpaper/{s2id}'
    rsp=requests.get(url,headers=headers)
    results=rsp.json()
    if 'error' in results:
        print(s2id)
        return None
    return results['recommendedPapers']

recs={}
for i in tqdm(refs_s2ids):
    s2id=refs_s2ids[i]
    rec=get_rec_papers(s2id)
    if rec is not None:
        recs[i]=rec
    time.sleep(0.1)

all_recs={}
for i in recs:
    for j in recs[i]:
        all_recs[j['paperId']]=j['title']

# all_recs_filtered={}
# for i in all_recs:
#     if i in refs_s2ids.values():
#         continue
#     if i in refs2_detail:
#         continue
#     all_recs_filtered[i]=all_recs[i]   


100%|██████████| 292/292 [02:03<00:00,  2.36it/s]


In [86]:
 def get_paper_detail(s2id):
    params={
        "fields": "title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,authors,embedding,tldr,openAccessPdf,externalIds,references",
    }
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results

save_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','plus')
refsp_detail={}
U.mkdir(save_dir)
for s2id in tqdm(all_recs_filtered):
    save_path=U.pjoin(save_dir,f'{s2id}.json')
    if U.pexists(save_path):
        refsp_detail[s2id]=U.load_json(save_path)
        continue
    ret=get_paper_detail(s2id)
    if 'error' not in ret:
        U.save_json(ret,save_path)
    refsp_detail[s2id]=ret
    time.sleep(0.1)


100%|██████████| 4409/4409 [00:41<00:00, 105.56it/s]


In [87]:
len(refsp_detail)

4409

In [92]:
len(all_recs_filtered)

refp_arxiv={}
refp_pdfs={}
for i in refsp_detail:
    if 'openAccessPdf' in refsp_detail[i] and refsp_detail[i]['openAccessPdf'] is not None and 'arxiv' not in refsp_detail[i]['openAccessPdf']['url']:
        url=refsp_detail[i]['openAccessPdf']
        refp_pdfs[i]=url['url']
        continue
    if 'externalIds' in refsp_detail[i]:
        if 'ArXiv' in refsp_detail[i]['externalIds']:
            refp_arxiv[i]=refsp_detail[i]['externalIds']['ArXiv']
            continue
    # print(i)
print(len(refp_arxiv))
print(len(refp_pdfs))

2803
214


In [94]:
ar5iv_url='https://ar5iv.labs.arxiv.org/html/{arxiv_id}'
htmlsp_dir=U.pjoin(ptree.lib_dir,'..','files','htmlsp')
U.mkdir(htmlsp_dir)
pdfsp_dir=U.pjoin(ptree.lib_dir,'..','files','pdfsp')
U.mkdir(pdfsp_dir)

    
# for i in tqdm(refp_arxiv):
#     DIR=U.pjoin(htmlsp_dir,f'{i}.html')
#     if os.path.exists(DIR):
#         continue
#     url=ar5iv_url.format(arxiv_id=refp_arxiv[i])

#     # Send a GET request to the URL
#     response = requests.get(url)

#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Save the HTML content to a file
#         with open(DIR, "w", encoding="utf-8") as file:
#             file.write(response.text)
#     else:
#         print(f"Failed to retrieve {i}. Status code: {response.status_code}")



failed_pdfs={}
for i in tqdm(refp_pdfs):
    url=refp_pdfs[i]
    save_path=U.pjoin(pdfsp_dir,f'{i}.pdf')
    if not os.path.exists(save_path):
        try:
            pdf=urllib.request.urlretrieve(url, save_path)
        except:
            failed_pdfs[i]=url
            print(f'Failed to download {i}, {url}')



  1%|          | 2/214 [00:00<00:15, 13.25it/s]

Failed to download 1edd8a9ce55c0696beeca75c5aa315b621a77359, https://dl.acm.org/doi/pdf/10.1145/3687475
Failed to download c738952caeb142be1d755ac6d50e7fbf5721b73b, https://www.mdpi.com/1424-8220/24/14/4733/pdf?version=1721635306
Failed to download 6ccd721263b09e13fdc93a919dd3e94de3fec783, https://www.mdpi.com/2079-9292/13/15/2928/pdf?version=1721897144
Failed to download a88fe2922556b2236bb53cea0240a1577d4d18f6, https://www.mdpi.com/2072-4292/16/14/2642/pdf?version=1721438473


  5%|▍         | 10/214 [00:22<08:53,  2.61s/it]

Failed to download 035f2dd4f219f67b6321c48931b00f2e3edf54ff, https://www.mdpi.com/2078-2489/15/9/517/pdf?version=1724574257
Failed to download 5c229896a16fc7b3ae040f1147982d98ee899df6, https://www.mdpi.com/2076-3417/14/15/6578/pdf?version=1722411871
Failed to download e763616e0bc236e8b23bb389dd837ebefbad32b7, https://dl.acm.org/doi/pdf/10.1145/3689632


  6%|▌         | 12/214 [00:23<06:16,  1.86s/it]

...

Failed to download b4eeebc419dc0394a64bd2991079cf978e5b2701, https://www.mdpi.com/2227-7390/12/16/2448/pdf?version=1723016818
Failed to download 74f6ea4ac45345d80768d2e2973b523f98b33f33, https://dl.acm.org/doi/pdf/10.1145/3688571


 98%|█████████▊| 210/214 [06:14<00:08,  2.01s/it]

Failed to download 516f48e222fdad35b92113ecdbf8a459d37c95aa, https://www.mdpi.com/2073-431X/13/7/176/pdf?version=1721618358
Failed to download a63b70ba0e65b4b2916093e15078348e8a5ec490, https://www.mdpi.com/2673-2688/5/3/74/pdf?version=1724902338


100%|██████████| 214/214 [06:18<00:00,  1.77s/it]

Failed to download 301aa759e431ae9c24b8deb274996c832bd20f0f, https://pubs.aip.org/aip/pop/article-pdf/doi/10.1063/5.0213625/20106149/082507_1_5.0213625.pdf


### Get references and files


In [34]:
params={
    "fields": "openAccessPdf,externalIds",
}

pdf_urls={}
external_ids={}
for i in tqdm(refs_s2ids):
    s2id=refs_s2ids[i]
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    if 'openAccessPdf' in results and results['openAccessPdf'] is not None:
        pdf_urls[i]=results['openAccessPdf']
    if 'externalIds' in results and results['externalIds'] is not None:
        external_ids[i]=results['externalIds']
    time.sleep(0.1)


arxiv_ids={}
for i,j in external_ids.items():
    if 'ArXiv' in j:
        arxiv_ids[i]=j['ArXiv']

arxiv_ids_add={
    'fire':'2310.04418',
    'mogrifier':'1909.01792',
    'hedgehog':'2402.04347',
    'synthesizer':'2005.00743',
    'settransformer':'1810.00825',
    'infiniteformer':'2109.00301',
    'etc':'2004.08483',
    'feedbackmem':'2002.09402',
    'kangpt':'2408.10205',
}

arxiv_ids.update(arxiv_ids_add)

ar5iv_url='https://ar5iv.labs.arxiv.org/html/{arxiv_id}'

100%|██████████| 292/292 [00:53<00:00,  5.42it/s]


In [7]:
def get_node(id):
    return ptree.G.nodes[id]['data']


def get_references(id):
    params={
        "fields": "references",
    }
    s2id=refs_s2ids[id]
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results['references']


all_refs={}
for i in tqdm(refs_s2ids):
    all_refs[i]=get_references(i)

refs_set={}
for i in all_refs:
    for ref in all_refs[i]:
        refs_set[ref['paperId']]=ref['title']


100%|██████████| 292/292 [00:33<00:00,  8.79it/s]


In [8]:
def get_paper_detail(s2id):
    params={
        "fields": "title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,authors,tldr,embedding,tldr,openAccessPdf,externalIds,references",
    }
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results

refs_detail={}
save_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','refs2')
for i in tqdm(refs_set):
    save_path=U.pjoin(save_dir,f'{i}.json')
    if U.pexists(save_path):
        refs_detail[i]=U.load_json(save_path)
        continue
    ret=get_paper_detail(i)
    if 'error' not in ret:
        U.save_json(ret,save_path)
    refs_detail[i]=ret
    time.sleep(0.1)


100%|██████████| 4775/4775 [00:21<00:00, 218.05it/s] 


In [72]:
refs2_detail={}
ref2_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','secondary')
for i in os.listdir(ref2_dir):
    refs2_detail[i.split('.')[0]]=U.load_json(U.pjoin(ref2_dir,i))

ref2_arxiv={}
ref2_pdfs={}
for i in refs_detail:
    if 'externalIds' in refs2_detail[i]:
        if 'ArXiv' in refs2_detail[i]['externalIds']:
            ref2_arxiv[i]=refs2_detail[i]['externalIds']['ArXiv']
            continue
    if 'openAccessPdf' in refs2_detail[i] and refs2_detail[i]['openAccessPdf'] is not None and 'arxiv' not in refs2_detail[i]['openAccessPdf']['url']:
        url=refs_detail[i]['openAccessPdf']
        ref2_pdfs[i]=url['url']
        continue
    # print(i)

print(len(ref2_arxiv))
print(len(ref2_pdfs))

In [25]:
import urllib, urllib.request

save_dir=U.pjoin(ptree.lib_dir,'..','pdfs2')
arxiv_pdf='https://arxiv.org/pdf/{arxiv_id}'
U.mkdir(save_dir)

# bar=tqdm(ref2_arxiv)
# for i in bar:
#     bar.set_description(f'Downloading {i}')
#     arxiv_id=ref2_arxiv[i]
#     download_arxiv_pdf(arxiv_id,save_dir,f'{i}.pdf')

failed_pdfs={}
for i in tqdm(ref2_pdfs):
    url=ref2_pdfs[i]
    save_path=U.pjoin(save_dir,f'{i}.pdf')
    if not os.path.exists(save_path):
        try:
            pdf=urllib.request.urlretrieve(url, save_path)
        except:
            failed_pdfs[i]=url
            print(f'Failed to download {i}, {url}')


  0%|          | 3/1108 [00:03<17:08,  1.07it/s]  

Failed to download 0048b9b8fd2c49a141a8f91168b7f0a30161c780, https://www.mdpi.com/1424-8220/23/16/7222/pdf?version=1692262936
Failed to download 006fdeff6e1a81c404317ee4056d6cc72f9c0e50, https://www.aclweb.org/anthology/P18-1208.pdf


  1%|          | 10/1108 [00:07<08:40,  2.11it/s]

Failed to download 013eb12ce5468f79d58bf859653f4929c5a2bd14, https://direct.mit.edu/tacl/article-pdf/doi/10.1162/tacl_a_00542/2074871/tacl_a_00542.pdf
Failed to download 0269e74017b160ef3a0a49dff6f6c27a8adf93cf, https://www.aclweb.org/anthology/D17-1014.pdf
Failed to download 027beed800f7d5e20194caf6d689345045e8d0d4, https://dl.acm.org/doi/pdf/10.1145/3219819.3219944


  2%|▏         | 17/1108 [00:07<04:10,  4.35it/s]

...

Failed to download fe5b2664159211f7a1a6cd959ef20dacae7175d8, https://www.aclweb.org/anthology/D18-1404.pdf
Failed to download fed1a48da3d35694da6a75952829fbce5ea232c9, https://www.aclweb.org/anthology/N15-1080.pdf


100%|█████████▉| 1103/1108 [21:11<00:01,  2.76it/s]

Failed to download ffa3810fb02bafe60b706c5ee11c185f5519afab, https://ieeexplore.ieee.org/ielx7/6287639/8948470/09123901.pdf
Failed to download ffb949d3493c3b2f3c9acf9c75cb03938933ddf0, https://www.aclweb.org/anthology/D17-1215.pdf


100%|██████████| 1108/1108 [21:16<00:00,  1.15s/it]

Failed to download fff100f13441b922c32b43f5628cdca285ec72a1, https://www.aclweb.org/anthology/D18-1208.pdf


In [28]:
import requests

save_dir=U.pjoin(ptree.lib_dir,'..','htmls2')
U.mkdir(save_dir)
ar5iv_url='https://ar5iv.labs.arxiv.org/html/{arxiv_id}'

for i in tqdm(ref2_arxiv):
    DIR=U.pjoin(save_dir,f'{i}.html')
    if os.path.exists(DIR):
        continue
    url=ar5iv_url.format(arxiv_id=ref2_arxiv[i])
        
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Save the HTML content to a file
        with open(DIR, "w", encoding="utf-8") as file:
            file.write(response.text)
        # print(f"HTML content saved to {i}.html")
    else:
        print(f"Failed to retrieve {i}. Status code: {response.status_code}")


100%|██████████| 3172/3172 [10:32<00:00,  5.02it/s]


In [46]:
htmls_dir=U.pjoin(ptree.lib_dir,'..','htmls')
pdfs_dir=U.pjoin(ptree.lib_dir,'..','pdfs')

htmls=[i.split('.')[0] for i in os.listdir(htmls_dir)]
pdfs=[i.split('.')[0] for i in os.listdir(pdfs_dir)]

print(len(htmls))
print(len(pdfs))

arxiv_url='https://arxiv.org/abs/{arxiv_id}'

for i in pdfs:
    if i not in htmls:
        if i in arxiv_ids:
            arxiv_id=arxiv_ids[i]
            url=ar5iv_url.format(arxiv_id=arxiv_id)
            arxiv=arxiv_url.format(arxiv_id=arxiv_id)
            print(url,arxiv,i)


281
297
https://ar5iv.labs.arxiv.org/html/1911.04070 https://arxiv.org/abs/1911.04070 bptrans
https://ar5iv.labs.arxiv.org/html/2302.04542 https://arxiv.org/abs/2302.04542 eva
https://ar5iv.labs.arxiv.org/html/2312.06635 https://arxiv.org/abs/2312.06635 gla
https://ar5iv.labs.arxiv.org/html/2407.09941 https://arxiv.org/abs/2407.09941 hydra
https://ar5iv.labs.arxiv.org/html/2302.10866 https://arxiv.org/abs/2302.10866 hyena
https://ar5iv.labs.arxiv.org/html/2310.18780 https://arxiv.org/abs/2310.18780 hyenadistill
https://ar5iv.labs.arxiv.org/html/2405.21060 https://arxiv.org/abs/2405.21060 mamba2
https://ar5iv.labs.arxiv.org/html/2407.10347 https://arxiv.org/abs/2407.10347 mamba4gcn
https://ar5iv.labs.arxiv.org/html/2310.01655 https://arxiv.org/abs/2310.01655 polysketchformer


In [78]:
save_dir

'c:\\ChengJunyan1\\Research\\model_discovery\\notebooks\\..\\model_discovery\\model\\library\\tree\\pdfs'

In [57]:
import urllib, urllib.request
url = 'http://export.arxiv.org/api/query?search_query="{query}"&start=0&max_results=1'
title = ptree.G.nodes[refs[0]]['data'].title
encoded_title = urllib.parse.quote(title)
url=url.format(query=encoded_title)
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))


<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3D%223D-RPE%3A%20Enhancing%20Long-Context%20Modeling%20Through%203D%20Rotary%20Position%20Encoding%22%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query="3D-RPE: Enhancing Long-Context Modeling Through 3D Rotary Position Encoding"&amp;id_list=&amp;start=0&amp;max_results=1</title>
  <id>http://arxiv.org/api/WZ9KoGw6d+aii+09GWS4ZE43lX0</id>
  <updated>2024-09-11T00:00:00-04:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage>
</feed>



In [46]:
len(pdf_urls)

103

In [ ]:
import agents.search_utils 
importlib.reload(agents.search_utils)

from agents.search_utils import SuperScholarSearcher





# dev tools

In [46]:
import importlib
import sys
# sys.path.append('..')

import model_discovery.agents.search_utils
importlib.reload(model_discovery.agents.search_utils)

from model_discovery.agents.search_utils import SuperScholarSearcher, pwc_search_patched

sss=SuperScholarSearcher(ptree,evo_system.stream)

# sss._load_libs()

In [47]:
ret=sss.search_external('Transformer')
print(sss.pretty_print(ret))


Searching Semantic Scholar for "Transformer" from NeurIPS, ICML, ICLR, ACL, EMNLP, NAACL after 2010...
Searching arXiv for "Transformer" in cs.LG, cs.CL...
Searching Papers with Code for "Transformer"...


2024-09-12:12:27:55,199 INFO     [_client.py:1038] HTTP Request: GET https://paperswithcode.com/api/v1/search/?page=1&items_per_page=5&q=Transformer "HTTP/1.1 200 OK"


Found 19 related papers
*1. Spatial Transformer Networks*
Authors: Max Jaderberg, K. Simonyan, Andrew Zisserman, K. Kavukcuoglu
TL;DR: This work introduces a new learnable module, the Spatial Transformer, which explicitly allows the spatial manipulation of data within the network, and can be inserted into existing convolutional architectures, giving neural networks the ability to actively spatially transform feature maps.
Abstract: Convolutional Neural Networks define an exceptionally powerful class of models, but are still limited by the lack of ability to be spatially invariant to the input data in a computationally and parameter efficient manner. In this work we introduce a new learnable module, the Spatial Transformer, which explicitly allows the spatial manipulation of data within the network. This differentiable module can be inserted into existing convolutional architectures, giving neural networks the ability to actively spatially transform feature maps, conditional on the feat

[{'paper': {'id': 'swin-transformer-hierarchical-vision',
   'arxiv_id': '2103.14030',
   'nips_id': None,
   'url_abs': 'https://arxiv.org/abs/2103.14030v2',
   'url_pdf': 'https://arxiv.org/pdf/2103.14030v2.pdf',
   'title': 'Swin Transformer: Hierarchical Vision Transformer using Shifted Windows',
   'abstract': 'This paper presents a new vision Transformer, called Swin Transformer, that capably serves as a general-purpose backbone for computer vision. Challenges in adapting Transformer from language to vision arise from differences between the two domains, such as large variations in the scale of visual entities and the high resolution of pixels in images compared to words in text. To address these differences, we propose a hierarchical Transformer whose representation is computed with \\textbf{S}hifted \\textbf{win}dows. The shifted windowing scheme brings greater efficiency by limiting self-attention computation to non-overlapping local windows while also allowing for cross-windo